<div style="background-color:Maroon;">
   
<br>    
<p style="text-align:center">
    <a href="https://www.troy.edu/" target="_blank">
    <img src="https://www.troy.edu/_resources/images/troy-logo-svg.svg" width="200" alt="Troy University Logo"  /> 
    </a>
</p>
<br> 
<p style="text-align:center">
<h1 style="color:White;line-height:.2"><center>MSIR Capstone </center> </h1>
<h3 style="color:White;line-height:.2"><center>Allison Moore</center> </h3>
<h2 style="color:White;line-height:.8"><center>October 2022</center> </h2>  
</p>
</div>


<div style="background-color:Black;">
    <h6> </h6>   
<h3 style="color:White;line-height:1.3""><center>Administrative Tasks & Data Wrangling </center> </h3>

</div>


In [1]:
#Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#Define a function to normalize a column of data
def normal(col):
    temp=[]
    least=np.min(col)
    most=np.max(col)
    for i in col:
        n=((i)-least)/(most-least)
        temp.append(n)
    return temp

#Define a function to normalize an entire DataFrame
def normal_DF(df):
    normalized_df=df
    for colName, colValue in df.iteritems():
        normalized_df[colName]=normal(colValue)
    return normalized_df

In [ ]:
# Define a function to generate a confusion matrix 
# to identify True Positives, True Negatives, False Positives,
# and False Negatives during Machine Learning

def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6']); ax.yaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6'])

In [ ]:
# Two DataFrames will be utilized. Both will contain the same data.
# However, one DataFrame will utilize only numerical, normalized values
# while the second will maintain some categorical text, for ease of 
# understanding during Data Visualization.

# Read the numerical CSV file
df=pd.read_csv('Numerical_Data.csv')

#Normalize the numerical Data Frame
ndf=normal_DF(ndf)

#Average each variable across each unique value of Severity.
Temp=ndf.groupby('SEV').mean()
ind=pd.Series([0,1,2,3,4,5,6])
#Reindex the new DataFrame with the values of the Severity variable.
Avg_ndf=Temp.set_index(ind)
Avg_ndf['SEV']=Temp.index


# Read the multi-type categorical CSV file
cdf=pd.read_csv('Categorical_Data.csv')



<div style="background-color:Black;">
    <h4> </h4>   
<h1 style="color:White;line-height:1.2""><center>Exploratory Data Analysis </center> </h1>

</div>


In [ ]:
#Quality correlation matrix
k = 8 #number of variables for heatmap
cols = df.corr().nlargest(k, 'SEV')['SEV'].index
cm = df[cols].corr()
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, cmap = 'viridis')

In [ ]:
#Outliers
#Use on Normalized DF

l = df.columns.values
number_of_columns=8
number_of_rows = len(l)-1/number_of_columns
plt.figure(figsize=(number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(df[l[i]],color='green',orient='v')
    plt.tight_layout()

In [ ]:
#Skewness
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.distplot(df[l[i]],kde=True) 

<div style="background-color:Black;">
    <h4> </h4>   
<h1 style="color:White;line-height:1.2""><center> Data Visualization </center> </h1>

</div>

In [ ]:
#Plot the averaged olity for each value of severity
sns.catplot(x="PVV", y='SEV', hue="SEV", data=Avg_ndf, aspect = 2)
plt.ylabel("Severity",fontsize=20)
plt.xlabel("Polity",fontsize=20)
plt.show()

<div style="background-color:Black;">
    <h4> </h4>   
<h1 style="color:White;line-height:1.2""><center> Machine Learning </center> </h1>

</div>

In [ ]:
x=ndf[['PVV','INT','METH','ICT','DPPL','CPL','GFP']]
y=ndf['SEV']
transform = preprocessing.StandardScaler(x)
X=transform
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

In [ ]:
# Logistic Regression Model
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()

logreg_cv = GridSearchCV(lr, parameters,cv=10, iid=None)
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)
TA_Log= logreg_cv.best_estimator_.score(X_test, y_test)
print(TA_Log)

yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(y_test,yhat)

In [ ]:
#Decision Tree
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()
tree_cv=GridSearchCV(tree,parameters, cv=10)
tree_cv.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)
TA_Tree=tree_cv.best_estimator_.score(X_test, y_test)
print(TA_Tree)
y_tree = tree_cv.predict(X_test)
plot_confusion_matrix(y_test,y_tree)

In [ ]:
#SVM
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()

svm_cv=GridSearchCV(svm, parameters, cv=10)
svm_cv.fit(X_train, Y_train)
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)
TA_SVM= svm_cv.best_estimator_.score(X_train, Y_train)
print(TA_SVM)
y_svm=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,y_svm)


In [ ]:
#K Nearest Neighbors
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()
knn_cv=GridSearchCV(KNN, parameters, cv=10)
knn_cv.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy :",knn_cv.best_score_)
TA_KNN=knn_cv.best_estimator_.score(X_test, y_test)
print(TA_KNN)
y_Knn = knn_cv.predict(X_test)
plot_confusion_matrix(y_test,y_Knn)

In [ ]:
#Plot Model Accuracies to determine the best fit for the data
score={'SVM':TA_SVM, 'KNN': TA_KNN,'Decision Tree' :TA_Tree,'Logistic Regression': TA_Log}
scores=pd.DataFrame(data=score, index=[0])
scores.head()
scores.plot(kind='bar', figsize=(20, 15))

plt.xlabel('Model',fontsize=20)
plt.ylabel('Accuracy',fontsize=20) 
plt.title('Model Prediciton Accuracy',fontsize=20)

plt.show()